# Código Postal

### Bibliotecas

- **Requests** para conectar ao Open Street Map.
- **pprint** para manuseio dos dados.

In [ ]:
import requests
from pprint import pprint

In [ ]:
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

- Solicita o Código Postal e transfere para URL.

In [ ]:
# postcode = 'G42 9AY'
postcode = str(input('Código postal (Partida): ')).strip()
response = requests.get(f"{BASE_URL}&postalcode={postcode}")

Transforma a resposta no formato json, se falhar, retorna o conteúdo da resposta.

In [ ]:
try:
    data = response.json()
    print(data)
except:
    print("Error from server: " + str(response.content))

A partir da resposta em Json, é extraído a latitude e a longitude da localização informada.

In [ ]:
latitude = data[0].get('lat')
longitude = data[0].get('lon')

In [ ]:
postcode = str(input('Código postal (Chegada): ')).strip()
response = requests.get(f"{BASE_URL}&postalcode={postcode}&country=brazil")
data = response.json()

latitude2 = data[0].get('lat')
longitude2 = data[0].get('lon')

(latitude, longitude)

# Folium (Mapa)

### Bibliotecas

- **Folium:** Para mostrar o mapa

### Código

- Localização recebe latitude e longitude;
- a variável m recebe a localização no mapa;
- um marcador é adicionado na localização informada.

In [ ]:
import folium
import openrouteservice as ors

client = ors.client(key='5b3ce3597851110001cf6248c92503e4c87f494aa259b8682754547b')

saida = float(latitude), float(longitude)
chegada = float(latitude2), float(longitude2)
coords = [[saida], [chegada]]

m = folium.Map(location=location, width=1920, height=1080)
folium.Marker(saida, popup='Saída!').add_to(m)
folium.Marker(chegada, popup='Chegada!').add_to(m)

route = client.directions(coordinates=coords,
                          profile='driving-car',
                          format='geojson')

folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']], color="blue").add_to(m)

m

